<a href="https://colab.research.google.com/github/jkordonis/TropicalML/blob/main/SVHN_VGG_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import random
from torch.utils.data import Dataset
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import copy
import cvxpy as cp
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms


import numpy as np
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
transform = transforms.Compose([ transforms.ToTensor(),
                               transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
training_dataset = datasets.SVHN('./data_src', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN('./data_src', split='test', download=True, transform=transform)
batch_size=32
training_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=True)


Using downloaded and verified file: ./data_src/train_32x32.mat
Using downloaded and verified file: ./data_src/test_32x32.mat


In [ ]:
class VGG_like_Net(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG_like_Net, self).__init__()
        self.layer1= nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.layer2= nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3= nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4= nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))


        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4*4*128, 1024),
            nn.ReLU())
        self.fc_out= nn.Sequential(
            nn.Linear(1024, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc_out(out)
        return out

model = VGG_like_Net().to(device)

In [ ]:
model_path = '/content/drive/My Drive/SVHN_VGG.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<ipython-input-37-db368024d950>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
def training_batch(data, labels, model, criterion, optimizer):
    model.train()
    data = data.to(device)
    labels = labels.to(device)
    output = model(data)
    loss = criterion(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
with torch.inference_mode():
    def test_batch(data, labels, model, criterion):
        model.eval()
        data = data.to(device)
        labels = labels.to(device)
        output = model(data)
        loss = criterion(output, labels)
        return loss

In [ ]:
with torch.inference_mode():
    def accuracy(data, labels, model):
        model.eval()
        data = data.to(device)
        labels = labels.to(device)
        output = model(data)
        _, pred_labels = output.max(-1)
        correct = (pred_labels == labels)
        return correct.cpu().detach().numpy().tolist()

In [ ]:
W1 = model.fc[1].weight.data.clone().detach().cpu().numpy()
b1 = model.fc[1].bias.data.clone().detach().cpu().numpy()
W2 = model.fc_out[0].weight.data.clone().detach().cpu().numpy()
b2 = model.fc_out[0].bias.data.clone().detach().cpu().numpy()

In [ ]:
def Sample_Pair_of_outp_to_Divide(W1_mat,W2,b1):
  idx_Out_1, idx_Out_2 = random.sample(range(10), 2)

  Coef = W2[idx_Out_1,:]-W2[idx_Out_2,:]
  W_pos = Coef[Coef>=0,None]*W1[Coef>=0]
  W_neg = Coef[Coef<0,None]*W1[Coef<0]
  b_pos = Coef[Coef>=0]*b1[idx_Out_1]
  b_neg = Coef[Coef<0]*b1[idx_Out_2]
  return W_pos,W_neg,b_pos,b_neg

W_pos,W_neg,b_pos,b_neg=Sample_Pair_of_outp_to_Divide(W1,W2,b1)

##  Define the Division Function ##



In [ ]:
def Division_function(a_divident,b_divident,Iterations =5,m_q=2):
  #Initialization Process: Start with almost the same number of samples per term
  d=a_divident.shape[1]
  a_hat, b_hat=np.random.randn(m_q,d), np.zeros(m_q)
  for i in range(20):
    I_i = np.argmax((a_hat@X_sample.T+np.array([b_hat]).T), axis=0)
    b_hat=b_hat-np.array([(I_i==j).sum() for j in range(m_q)])*0.005
    b_hat=b_hat-b_hat.mean()

  #Compute the f values
  f_x_i=np.max((a_divident@X_sample.T+np.array([b_divident]).T), axis=0)
  m_p=a_divident.shape[0]
  # Define cp variables
  x_b, lambda_var =   cp.Variable(1), cp.Variable(m_p)
  for cnt in range(Iterations):
    I_i=np.argmax((a_hat@X_sample.T+np.array([b_hat]).T), axis=0)         # Find the sets I_i
    N=np.array([(I_i==j).sum() for j in range(m_q)])                      # Compute the number of elements in each set I_i
    s=np.array([np.sum(X_sample[I_i==i], axis=0) for i in range(m_q)])    # Compute the summation o x_j's
    for i in range(a_hat.shape[0]):
      # In this problem we substituted a_hat_i by a linear combination of a_i's
      prob = cp.Problem(cp.Maximize(lambda_var@a_divident@s[i]+N[i]*x_b),
                      [(X_sample@(lambda_var@a_divident))+x_b<=f_x_i,
                      lambda_var>=0,
                      cp.sum(lambda_var)==1,
                      ])
      prob.solve(warm_start=True)
      a_hat[i]=(lambda_var@a_divident).value
      b_hat[i]=x_b.value
  return a_hat,b_hat

In [ ]:
intermediate_output = None

def hook_fn(module, input, output):
    global intermediate_output
    intermediate_output = output

hook_handle = model.layer6.register_forward_hook(hook_fn)
X_sample=np.zeros([0,4*4*128])

cnt=0
for i in training_dataloader:
  inputs = i[0].to(device)
  output = model(inputs)
  X_to_add = intermediate_output.reshape(batch_size, -1).detach().to('cpu').numpy()
  X_sample=np.concatenate((X_sample,X_to_add),axis=0)
  cnt+=1
  if cnt>=7:
    break

X_sample = X_sample[:200]


In [ ]:
import time
T=time.time()

a_hat_pos_list,b_hat_pos_list,a_hat_neg_list,b_hat_neg_list=[],[],[],[]  # Initialize the lists
for number_of_terms in range(100):
  W_pos,W_neg,b_pos,b_neg=Sample_Pair_of_outp_to_Divide(W1,W2,b1)
  a_hat,b_hat=Division_function(W_pos,b_pos)
  a_hat_pos_list.append(a_hat)
  b_hat_pos_list.append(b_hat)
  a_hat,b_hat=Division_function(W_neg,b_neg)
  a_hat_neg_list.append(a_hat)
  b_hat_neg_list.append(b_hat)
  #print('iteration', number_of_terms,'Time',time.time()-T)



<ipython-input-23-82305a24b546>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b_hat[i]=x_b.value
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [ ]:
a_hat_np_array_pos = np.array(a_hat_pos_list)
b_hat_np_array_pos = np.array(b_hat_pos_list)
a_hat_np_array_neg = np.array(a_hat_neg_list)
b_hat_np_array_neg = np.array(b_hat_neg_list)

model_path_folder = '/content/drive/My Drive/'

np.save(model_path_folder+'SVHN_b_hat_np_array_pos.npy',b_hat_np_array_pos)
np.save(model_path_folder+'SVHN_a_hat_np_array_neg.npy',a_hat_np_array_neg)
np.save(model_path_folder+'SVHN_b_hat_np_array_neg.npy',b_hat_np_array_neg)


'/content/drive/My Drive/SVHN_a_hat_np_array_pos.npy'

In [ ]:
model_path_folder = '/content/drive/My Drive/'

a_hat_np_array_pos = np.load(model_path_folder+'SVHN_b_hat_np_array_pos.npy')
b_hat_np_array_pos = np.load(model_path_folder+'SVHN_b_hat_np_array_pos.npy')
a_hat_np_array_neg = np.load(model_path_folder+'SVHN_a_hat_np_array_neg.npy')
b_hat_np_array_neg = np.load(model_path_folder+'SVHN_b_hat_np_array_neg.npy')


FileNotFoundError: [Errno 2] No such file or directory: 'SVHN_a_hat_np_array_pos.npy'

# Simplified NN #

In [ ]:
Vectors_to_choose=np.zeros([0,2048])
Corresp_bias=np.zeros([0])
for list_var in range(len(a_hat_pos_list)):
  Vectors_to_choose=np.append(Vectors_to_choose, a_hat_pos_list[list_var], axis=0).copy()
  Corresp_bias=np.append(Corresp_bias, b_hat_pos_list[list_var], axis=0).copy()

  Vectors_to_choose=np.append(Vectors_to_choose, a_hat_neg_list[list_var], axis=0).copy()
  Corresp_bias=np.append(Corresp_bias, b_hat_pos_list[list_var], axis=0).copy()


-----------------a--------------

In [ ]:
K = 3
W_form_Division = Vectors_to_choose[:2*K]
W_form_Division = torch.from_numpy((W_form_Division)).float()
biass_from_Division = Corresp_bias[:2*K]
biass_from_Division = torch.from_numpy(biass_from_Division).float()


A_mat1 = torch.zeros(K, 2 * K).float()
A_mat2 = torch.zeros(K, 2 * K).float()

for i in range(K):
    A_mat1[i,2*i] = 1
    A_mat2[i,2*i+1] =  1

class small_nn_head(nn.Module):
    def __init__(self, number_of_tropical_terms=K):
        super(small_nn_head, self).__init__()

        # Define layers
        self.Layer1 = nn.Linear(4 * 4 * 128, 2 * number_of_tropical_terms)
        self.Lin_part1 = nn.Linear(2*number_of_tropical_terms, number_of_tropical_terms)
        self.Lin_part2 = nn.Linear(2*number_of_tropical_terms, number_of_tropical_terms)
        self.Layer2 = nn.Linear(3*number_of_tropical_terms,10)


        self.Lin_part1.weight.data = A_mat1
        self.Lin_part1.bias.data = torch.zeros(number_of_tropical_terms)
        self.Lin_part2.weight.data = A_mat2
        self.Lin_part2.bias.data = torch.zeros(number_of_tropical_terms)
        self.Layer1.weight.data = W_form_Division
        self.Layer1.bias.data = biass_from_Division


    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        out = self.Layer1(x)
        out1 = self.Lin_part1(out)
        out2 = self.Lin_part2(out)
        max_12 = torch.max(out1, out2)
        out = torch.cat((out, max_12), dim=1)
        out = self.Layer2(out)
        return out

# Create the model instance and move it to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_substitute = small_nn_head(number_of_tropical_terms=K)
model_substitute.to(device)

small_nn_head(
  (Layer1): Linear(in_features=2048, out_features=6, bias=True)
  (Lin_part1): Linear(in_features=6, out_features=3, bias=True)
  (Lin_part2): Linear(in_features=6, out_features=3, bias=True)
  (Layer2): Linear(in_features=9, out_features=10, bias=True)
)

In [ ]:
# Create Dataset for compressed Layer
activation = {}
def getActivation(name):
  def hook(model, input, output):
    activation[name] = output.detach()
  return hook

def set_hooks(INP_Layer):
  hook_INP_to_Compressed_Layer = INP_Layer.register_forward_hook(getActivation('INP_to_Compressed_Layer'))

class LayerComputeDataset(Dataset):
    def __init__(self, Init_model,from_layer):
        self.from_layer= from_layer
        self.Init_model=Init_model
        self.num_samples = training_dataset.data.shape[0]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        x= training_dataset.data[idx]
        with torch.no_grad():
#          self.Init_model(transforms.ToTensor()(x).unsqueeze(0).float().to(device))
          self.Init_model(torch.tensor(x,dtype=torch.float32).unsqueeze(0).float().to(device))

        input_data = activation['INP_to_Compressed_Layer']
        labels = training_dataset.labels[idx]

        return input_data.squeeze(), labels


In [ ]:
set_hooks( model.layer6)
DataSetForCompressLayer=LayerComputeDataset(model,'INP_to_Compressed_Layer')
DataLoaderForCompressLayer = DataLoader(DataSetForCompressLayer, batch_size=32, shuffle=True,num_workers=0)

In [ ]:
class VGG_like_Net_COMPRESSED(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG_like_Net_COMPRESSED, self).__init__()
        self.layer1= nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.layer2= nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3= nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer4= nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))


        self.fc = small_nn_head(number_of_tropical_terms=K)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

Model_Compressed = VGG_like_Net_COMPRESSED().to(device)

In [ ]:
Model_Compressed.layer1 = copy.deepcopy(model.layer1)
Model_Compressed.layer2 = copy.deepcopy(model.layer2)
Model_Compressed.layer3 = copy.deepcopy(model.layer3)
Model_Compressed.layer4 = copy.deepcopy(model.layer4)
Model_Compressed.layer5 = copy.deepcopy(model.layer5)
Model_Compressed.layer6 = copy.deepcopy(model.layer6)



In [ ]:
sum(p.numel() for p in Model_Compressed.parameters())/sum(p.numel() for p in model.parameters())

0.12533332220520546

In [ ]:
model_to_train=model_substitute
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_to_train.parameters(), lr=0.001)

n_epochs = 5

for epoch in range(n_epochs):
    training_losses, test_losses = [], []
    training_accuracies, test_accuracies = [], []
    for data, labels in DataLoaderForCompressLayer:
        trng_batch_loss = training_batch(data, labels, model_to_train, criterion, optimizer)
        training_losses.append(trng_batch_loss.item())
        trng_batch_correct = accuracy(data, labels, model_to_train)
        training_accuracies.extend(trng_batch_correct)
    training_per_epoch_loss = np.array(training_losses).mean()
    training_per_epoch_accuracy = np.mean(training_accuracies)
    mean_loss =[]
    Model_Compressed.fc = copy.deepcopy(model_substitute)
    for data, labels in test_dataloader:
      TruthValue=torch.argmax(Model_Compressed(data.to(device)),axis=1)==labels.to(device)
      mean_loss.append(TruthValue.sum()/labels.shape[0])

    print(f'Epoch: {epoch+1}/{n_epochs}\t| Training loss: {training_per_epoch_loss:.4f} | ', end='')
    print(f'Test accuracy: {torch.tensor(mean_loss).mean():.4f}')

Epoch: 1/5	| Training loss: 0.8072 | Test accuracy: 0.8915
Epoch: 2/5	| Training loss: 0.3280 | Test accuracy: 0.9085
Epoch: 3/5	| Training loss: 0.2659 | Test accuracy: 0.9172
Epoch: 4/5	| Training loss: 0.2394 | Test accuracy: 0.9209
Epoch: 5/5	| Training loss: 0.2225 | Test accuracy: 0.9244


In [ ]:
model_copy = copy.deepcopy(model)
W1 = model_copy.fc[1].weight.detach()
b1 = model_copy.fc[1].bias.detach()
N=int(1024*(1-))
NORMS=torch.tensor([torch.norm(W1[i]) for i in range(1024)])
values, indices = torch.topk(NORMS, N, largest=False)
W1[indices]=0
b1[indices]=0


model_copy.fc[1].weight.data.copy_(W1)
model_copy.fc[1].bias.data = b1.clone()
for data, labels in test_dataloader:
      TruthValue=torch.argmax(model_copy(data.to(device)),axis=1)==labels.to(device)
      mean_loss.append(TruthValue.sum()/labels.shape[0])
print(torch.tensor(mean_loss).mean())

tensor(0.9207)


In [ ]:
(torch.tensor([torch.norm(W1[i]) for i in range(1024)])==0).sum()


tensor(921)